This is a getting started guide to help deploy MLflow locally. 

If you are a machine learning developer like me, you typically spend most of your time experimenting with parameters and code on your local workstation using your favorite IDE or Jupyter Notebook. In the past, I would use a spreadsheet to track my ML experiments. That was not fun.  

The ML stack was begging for a toolchain that can bring DevOps-like functionality to ML development.  This led to the rise of tools referred to loosely as MLOps tools. One of those tools that's gaining traction is MLflow.

MLflow is an open source framework to help organize machine learning development workflows by tracking experiments, packaging code into reproducible executions, facilitating collaboration, and deploying models. 

MLflow can be deployed in many configurations. If you are developing locally most of the time and don't have the need for a central repo MLflow can be installed locally on your laptop so you can track your experiments and never have to guess again if you tried certain parameters or not and what was the resulting ML model performance. 

In this guide I will walk you thru how I deployed MLflow on my local macbook. Later, I will publish guides on more advanced setups such as using a cloud-hosted resources to support team collaboration and for higher availability, scalability, and security. 

LET'S GET STARTED!


In [1]:
# Install mlflow

!pip install mlflow

In [2]:
# Import support libraries 

import os
import warnings
import sys

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn

In [3]:
import logging

# configure logging

logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)

In [4]:
# define evaluation metrics. In this case rmse, mae, r2 scores

def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

In [5]:
#configure warning logs
warnings.filterwarnings("ignore")

# set random seed to reproduce same results everytime 

np.random.seed(40)

# Read the samle dataset csv file from the URL
csv_url = (
    "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
)
try:
    data = pd.read_csv(csv_url, sep=";")
except Exception as e:
    logger.exception(
        "Unable to download training & test CSV, check your internet connection. Error: %s", e
    )
    
# Split the data into training and test sets. (0.75, 0.25) split.
train, test = train_test_split(data)

# The predicted column is "quality" which is a scalar from [3, 9]
train_x = train.drop(["quality"], axis=1)
test_x = test.drop(["quality"], axis=1)
train_y = train[["quality"]]
test_y = test[["quality"]]



In [6]:
# Run this code cell multiple times each with a different alpha and l1_ratio numbers using 
# values between 0 and 1. 

# ElasticNet link: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html

alpha =  .35 # used to multiply penalty terms.
l1_ratio =  .45  # a penalty function var
# This example uses the ElasticNet model. 
# It's a linear regression model with combined L1 and L2 priors as regularizer

with mlflow.start_run():
    lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
    lr.fit(train_x, train_y)

    predicted_qualities = lr.predict(test_x)

    (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

    print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
    print("  RMSE: %s" % rmse)
    print("  MAE: %s" % mae)
    print("  R2: %s" % r2)

    mlflow.log_param("alpha", alpha)
    mlflow.log_param("l1_ratio", l1_ratio)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)
    mlflow.log_metric("mae", mae)

    tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

    # Model registry does not work with file store
    if tracking_url_type_store != "file":

        # Register the model
        # There are other ways to use the Model Registry, which depends on the use case,
        # please refer to the doc for more information:
        # https://mlflow.org/docs/latest/model-registry.html#api-workflow
        mlflow.sklearn.log_model(lr, "model", registered_model_name="ElasticnetWineModel")
    else:
        mlflow.sklearn.log_model(lr, "model")


# for each run, above metrics will be saved in model's local directory. 

Elasticnet model (alpha=0.350000, l1_ratio=0.450000):
  RMSE: 0.7616514499663437
  MAE: 0.5936841528680933
  R2: 0.17804834226795552


In [7]:
# when done, open a terminal and go to your projects local working 
# directory and below command (withour hashtag)

# mlflow ui

In [8]:
# from your browser, open the mflow dashboard using below url (withour hashtag)

# http://localhost:5000
        
# you should see a dashboard similar to the one below but your content will vary.

# ![title](img/mbitar-mflow-dash.jpg)

![title](img/mbitar-mflow-dash.jpg)

In [ ]:
# when you click on one of the experiment (runs) you can see more details such as related artifacts
#  ![title](img/mbitar-mflow-dash.jpg)


![title](img/mbitar-mflow-dash-details.jpg)

In [ ]:
# you can also track artifact metadata such as model matadata 

![mflow dash 1](https://github.com/scriptbuzz/mlflow-local/blob/main/mbitar-mflow-dash-artifacts.jpg)

mlflow is feature-packed. If you are not using an MLops tool, you can enjoy substantial gains in productivity by simply by using it's basic features. Once you feel comfortable with the basics, you can exapnd your MLflow deployment. 

My next MLops guide will cover remote MLflow deployment and touch on some of the tool's more advanced features to support cloud and team collaboration. 

Thank you.
Michael Bitar